In [1]:
import torch

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
X, y = load_iris(as_frame=True, return_X_y=True)

In [4]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Tensor -> Numpy Nd-Array

In [ ]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_train

In [9]:
X_train.dtype

torch.float32

In [13]:
X_train.shape[0]

105

In [12]:
X_train.size(0)

105

In [17]:
X_test = torch.from_numpy(X_test.values)
X_test.dtype

torch.float64

In [18]:
y_train, y_test = torch.from_numpy(y_train.values), torch.tensor(y_test.values)

Dataset

In [ ]:
X_train[0:7]

In [19]:
class IrisDataset(torch.utils.data.Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.size(0) # self.X.shape[0]

  def __getitem__(self, idx): # idx -> [0, 1, 2, 3]
    return self.X[idx, :], self.y[idx]

In [20]:
train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)

In [26]:
train_dataset[:5]

(tensor([[5.5000, 2.4000, 3.7000, 1.0000],
         [6.3000, 2.8000, 5.1000, 1.5000],
         [6.4000, 3.1000, 5.5000, 1.8000],
         [6.6000, 3.0000, 4.4000, 1.4000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([1, 2, 2, 1, 2]))

Dataloader

In [34]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=20)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=20)

In [35]:
for batch in train_dataloader:
  print(batch[0].size(), batch[1].size())


torch.Size([20, 4]) torch.Size([20])
torch.Size([20, 4]) torch.Size([20])
torch.Size([20, 4]) torch.Size([20])
torch.Size([20, 4]) torch.Size([20])
torch.Size([20, 4]) torch.Size([20])
torch.Size([5, 4]) torch.Size([5])


Model

In [38]:
class IrisClassifier(torch.nn.Module):
  def __init__(self, in_dims, n_classes):
    super(IrisClassifier, self).__init__()

    self.in_dims = in_dims
    self.n_classes = n_classes

    # Model Layers:
    self.input_layer = torch.nn.Linear(self.in_dims, 64)
    self.hidden_layer = torch.nn.Linear(64, 32)
    self.output_layer = torch.nn.Linear(32, self.n_classes)

    # Activations:
    self.relu = torch.nn.ReLU()
    self.softmax = torch.nn.Softmax(1)


  def forward(self, x_batch):
    out = self.relu(self.input_layer(x_batch))
    out = self.relu(self.hidden_layer(out))
    out = self.softmax(self.output_layer(out))

    return out


Training Loop

In [39]:
model = IrisClassifier(X_train.size(1), 3)

In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [41]:
EPOCHS = 20

for epoch in range(EPOCHS): # Epochs
  for batch_x, batch_y in train_dataloader:  # Iteration

    model.train() # model.eval()

    # Clear Gradient
    optimizer.zero_grad()

    # Forward Step
    output_train = model(batch_x)

    # Compute Loss
    loss_train = criterion(output_train, batch_y.long())

    # Backward Step
    loss_train.backward()

    # Update Model Weights
    optimizer.step()

    with torch.no_grad():
      output_test = model(X_test.float())
      loss_test = criterion(output_test, y_test.long())

  if (epoch+1)%5==0:
    print(f"Epoch: {epoch+1} - Train Loss: {loss_train.item()} - Test Loss: {loss_test.item()}")


Epoch: 5 - Train Loss: 0.6849881410598755 - Test Loss: 0.6761581897735596
Epoch: 10 - Train Loss: 0.5648865699768066 - Test Loss: 0.5970046520233154
Epoch: 15 - Train Loss: 0.5549188852310181 - Test Loss: 0.585442066192627
Epoch: 20 - Train Loss: 0.55291348695755 - Test Loss: 0.5824418663978577


Prediction

In [42]:
import numpy as np
model.eval()
with torch.no_grad():
  y_pred = np.argmax(model(X_test.float()), 1)

In [43]:
y_pred

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0])

In [44]:
from sklearn.metrics import classification_report

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

